In [12]:
import gymnasium as gym
import torch
import torch.nn as nn
import numpy as np
from collections import deque
import random

In [13]:
env=gym.make('MountainCar-v0')

In [14]:
env.observation_space

Box([-1.2  -0.07], [0.6  0.07], (2,), float32)

In [15]:
env.reset()


(array([-0.43698287,  0.        ], dtype=float32), {})

In [16]:

class NN(nn.Module):
    def __init__(self, input_dim, num_actions):
        super(NN, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 24),
            nn.ReLU(),
            nn.Linear(24, 48),
            nn.ReLU(),
            nn.Linear(48, num_actions),
            nn.Softmax()
        )

    def forward(self, x):
        return self.layers(x)

class DQN:
    def __init__(self, env, epsilon):
        self.env = env
        self.num_actions = env.action_space.n
        self.max_buffer_size = 250
        self.buffer = deque(maxlen=self.max_buffer_size)
        self.Q = NN(2, 3)
        self.Q_hat = NN(2, 3)
        self.optimizer = torch.optim.Adam(self.Q.parameters(), lr=0.001)
        self.epsilon = epsilon
        self.C = 32

    def add_in_buffer(self, state, action, reward, next_state, terminal):
        self.buffer.append((state, action, reward, next_state, terminal))

    def select_action(self, q_val):
        if random.random() < self.epsilon:
            return random.randint(0, self.num_actions - 1)
        else:
            return torch.argmax(q_val).item()

    def epsilon_decay(self, decay):
        self.epsilon = self.epsilon * decay

    def train(self, num_episodes):
        batch_size = self.max_buffer_size // 3
        for i in range(num_episodes):
            print(i)
            state = env.reset()
            state = self.state_to_float(state)
            self.epsilon_decay(0.5)
            terminal = False
            for _ in range(200):
                actions = self.Q.forward(torch.tensor(state).float())
                action = self.select_action(actions)
                next_state, reward, terminal, truncated, _ = self.env.step(action)
                if truncated==True:
                    break
                self.add_in_buffer(state, action, reward, next_state, terminal)
                state = next_state

                if len(self.buffer) < batch_size:
                    if terminal:
                        break
                    continue
                else:
                    sampled_batch = random.sample(self.buffer, batch_size)
                    targets = []
                    q_s_a = []
                    for i in range(batch_size):
                        state1, action1, reward1, next_state1, terminal1 = sampled_batch[i]
                        max_Q_s_a = torch.max(self.Q_hat.forward(torch.tensor(next_state1).float()).detach())
                        q_s_a.append(self.Q.forward(torch.tensor(state1).float())[action1])
                        targets.append(reward1 + max_Q_s_a * (1 - terminal1))

                    loss = nn.functional.mse_loss(torch.stack(targets), torch.stack(q_s_a))
                    self.optimizer.zero_grad()
                    loss.backward()
                    self.optimizer.step()

                if terminal:
                    break

    def avg_reward(self, num_episodes):
        tot_reward = 0
        for _ in range(num_episodes):
            state = env.reset()
            state = self.state_to_float(state)
            self.epsilon_decay(1 / (num_episodes + 1))
            terminal = False
            for _ in range(200):
                actions = self.Q.forward(torch.tensor(state).float())
                action = torch.argmax(actions).item()
                next_state, reward, terminal, truncated, _ = self.env.step(action)
                if truncated==True:
                    break
                tot_reward += reward
                state = next_state
                if terminal:
                    break
        return tot_reward / num_episodes

    def state_to_float(self, state):
        return state[0]


In [17]:
model=DQN(env,1)
print(model.avg_reward(100))
model.train(200) 

c:\Python\Lib\site-packages\torch\nn\modules\container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


-199.0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [18]:
model.avg_reward(1000)

-199.0